In [1]:
import os
import openai
from typing import List
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain.agents import tool
import requests
import datetime
from langchain.tools.render import format_tool_to_openai_function
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
import wikipedia
from langchain.prompts import MessagesPlaceholder
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.agent import AgentFinish
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory
import param
import json

from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
### Functions

class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
        'temperature_unit': 'fahrenheit',
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°F'

@tool
def lights_on(query: str) -> str:
    """This tools turns the lights on. 
    This command will be used if a user asks to turn on or off the lights.
    If this returns True, the response back will ALWAYS be 'The lights are on'
    If returns false, the response will be 'The lights are off'
    The Query arguments must either be: "turn on the lights" or "turn off the lights"
    """
    if query == 'turn on the lights':
        return f"Lights are on is: True"
    else:
        return f"Lights are off is: True"

In [9]:
from utils.scrape_method import *

def scrape_news() -> str:
    """
    Scrape news from the OCC website.
    Any request that is specifically asking about a person, place, or thing will be searched on Wikipedia.
    I want you to only mention the title of the news article. 
    Keep the link but do not read it in the response.
    """
    existing_links = load_existing_links()
    new_links =[]
    if not os.path.exists('utils/links.csv'):
        with open('links.csv', 'w', newline='', encoding='utf-8') as csvfile:  # Just write headers
            writer = csv.writer(csvfile)
            writer.writerow(["Title", "Link", "Source"])

    scraper = WebScraper(OccGovScraper())
    occ_links = scraper.execute_scrape()
    new_occ_links = get_new_links(existing_links, occ_links)
    
    if new_occ_links:
        print("\033[1m\033[94mNew OCC Links:\033[0m")
        news_dict = {}
        for title, link in new_occ_links:
            news_dict[title] = link
            print(f"{title}: {link}")
        write_to_csv(new_occ_links, scraper.get_source_name())
        return news_dict
    else:
        print("\033[1m\033[94mNo new OCC links found.\033[0m")
        return "No new OCC links found."

In [11]:
scrape_news()

New OCC Links:
OCC Hosts Public Appraisal Subcommittee Hearing on Appraisal Bias: https://www.occ.gov/news-events/events/files/event-other-event-occ-hearing-appraisal-bias.html


{'OCC Hosts Public Appraisal Subcommittee Hearing on Appraisal Bias': 'https://www.occ.gov/news-events/events/files/event-other-event-occ-hearing-appraisal-bias.html'}

In [8]:
type(scrape_news())

No new OCC links found.


str